## Import dependencies

In [1]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

## Import configuration variables

In [2]:
# import sys
# sys.path.append('../../../../')
from sfconfig import sf_username, sf_password, sf_security_token
from sfconfig import remote_db_endpoint, remote_db_port
from sfconfig import remote_db_name, remote_db_user, remote_db_pwd

## Set up `simple_salesforce`

Pass in the username, password, and security token here. The security token contains information about your org.

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [51]:
course_data = pd.read_sql("SELECT * FROM course", conn)
class_data = pd.read_sql("SELECT * FROM class", conn)
staff_data = pd.read_sql("SELECT * FROM staff", conn)
staffassignment_data = pd.read_sql("SELECT * FROM staffassignment", conn)
classparticipant_data = pd.read_sql("SELECT * FROM classparticipant", conn)
student_data = pd.read_sql("SELECT * FROM student", conn)

In [6]:
staff_data.head()

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


## Salesforce Population

Course Data and Class Data

In [14]:
for index, row in course_data.iterrows():

    record = {"ID_Course__c": row["ID_Course"],
              "CourseCode__c": row["CourseCode"],
              "CreditHours__c": row["CreditHours"],
              "CourseName__c": row["CourseName"],
              "BootCampCourse__c": row["BootCampCourse"]}
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

In [54]:
sf_course_records = sf.query("SELECT id, name, CourseName__c, ID_Course__c FROM Course__c")
coursedict = {}
for x in sf_course_records['records']:
    coursedict[x['ID_Course__c']] = {'name': x['CourseName__c'], 'id':x['Id']}

In [48]:
class_data['StartDate'] = class_data['StartDate'].apply(lambda x: x.strftime('%Y-%m-%d'))
class_data['EndDate'] = class_data['EndDate'].apply(lambda x: x.strftime('%Y-%m-%d')) 
class_data

   ID_Class  ID_Course               Section   StartDate     EndDate
0         1          1  GWU-ARL-DATA-PT-09-0  2020-03-16  2020-03-14
1         2          1       GWDC201805DATA3  2018-05-15  2018-11-08
2         3          2       GWARL201905WEB3  2019-05-14  2019-11-07
3         4          3      GWARL201905UIUX3  2019-05-14  2019-11-07


In [55]:
for index, row in class_data.iterrows():

    record = {"ID_Class__c": row["ID_Class"],
              "ID_Course__c": classdict[row["ID_Course"]]['id'],
              "Section__c": row["Section"],
              "StartDate__c": row["StartDate"],
              "EndDate__c": row["EndDate"]}
    
    try:
        sf.class__c.create(record)
    except Exception as e:
        print(e)

Staff Data and Staff Assignment Data

In [7]:
staff_data['BirthDate'] = staff_data['BirthDate'].apply(lambda x: x.strftime('%Y-%m-%d'))
print(staff_data)

   ID_Staff EmployeeID  LastName  FirstName MiddleName   BirthDate
0         1  000184220  Wimberly        Sam       Nico  1995-07-05
1         2  000130109   Sanford     Gemini      Blair  1992-04-22
2         3  000160655  Williams  Dartanion  De Angelo  1993-05-21
3         4  000159108   Popelka      Sarah      Nisan  1994-06-07


In [9]:
for index, row in staff_data.iterrows():

    record = {"ID_Staff__c": row["ID_Staff"],
              "EmployeeID__c": row["EmployeeID"],
              "LastName__c": row["LastName"],
              "FirstName__c": row["FirstName"],
              "MiddleName__c": row["MiddleName"],
              "BirthDate__c": row["BirthDate"]}
    
    try:
        sf.staff__c.create(record)
    except Exception as e:
        print(e)

In [12]:
sf_staff_records = sf.query("SELECT id, ID_Staff__c, FirstName__c, LastName__c FROM staff__c")
staffdict = {}
for x in sf_staff_records['records']:
    staffdict[x['ID_Staff__c']] = {'FirstName': x['FirstName__c'], 'LastName': x['LastName__c'],'id':x['Id']}

In [24]:
sf_class_records = sf.query("SELECT id, ID_Class__c, Section__c FROM class__c")
classdict = {}
for x in sf_class_records['records']:
    classdict[x['ID_Class__c']] = {'Section': x['Section__c'], 'id':x['Id']}

In [25]:
print(staffdict)
print()
print(classdict)

{1.0: {'FirstName': 'Sam', 'LastName': 'Wimberly', 'id': 'a063h000003cR6XAAU'}, 2.0: {'FirstName': 'Gemini', 'LastName': 'Sanford', 'id': 'a063h000003cR6cAAE'}, 3.0: {'FirstName': 'Dartanion', 'LastName': 'Williams', 'id': 'a063h000003cR6hAAE'}, 4.0: {'FirstName': 'Sarah', 'LastName': 'Popelka', 'id': 'a063h000003cR6mAAE'}}

{2.0: {'Section': 'GWDC201805DATA3', 'id': 'a083h0000016LjOAAU'}, 1.0: {'Section': 'GWU-ARL-DATA-PT-09-0', 'id': 'a083h0000016LjJAAU'}, 3.0: {'Section': 'GWARL201905WEB3', 'id': 'a083h0000016LjTAAU'}, 4.0: {'Section': 'GWARL201905UIUX3', 'id': 'a083h0000016LjAAAU'}}


In [22]:
staffassignment_data['StartDate'] = staffassignment_data['StartDate'].apply(lambda x: x.strftime('%Y-%m-%d'))
staffassignment_data

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [26]:
for index, row in staffassignment_data.iterrows():

    record = {"ID_StaffAssignment__c": row["ID_StaffAssignment"],
              "ID_Staff__c": staffdict[row["ID_Staff"]]['id'],
              "ID_Class__c": classdict[row["ID_Class"]]['id'],
              "Role__c": row["Role"],
              "StartDate__c": row["StartDate"],
              "EndDate__c": row["EndDate"],}
    
    try:
        sf.Staff_Assignment__c.create(record)
    except Exception as e:
        print(e)

Student Data and Class Participant Data


In [32]:
for index, row in student_data.iterrows():

    record = {"ID_Student__c": row["ID_Student"],
              "StudentID__c": row["StudentID"],
              "LastName__c": row["LastName"],
              "FirstName__c": row["FirstName"],
              "MiddleName__c": row["MiddleName"],
              "BirthDate__c": row["BirthDate"],
              "Gender__c": row["Gender"]}
    
    try:
        sf.student__c.create(record)
    except Exception as e:
        print(e)

In [52]:
sf_student_records = sf.query("SELECT id, ID_Student__c, FirstName__c, LastName__c FROM student__c")
studentdict = {}
for x in sf_student_records['records']:
    studentdict[x['ID_Student__c']] = {'FirstName': x['FirstName__c'], 'LastName': x['LastName__c'],'id':x['Id']}

In [53]:
sf_class_records = sf.query("SELECT id, ID_Class__c, Section__c FROM class__c")
classdict = {}
for x in sf_class_records['records']:
    classdict[x['ID_Class__c']] = {'Section': x['Section__c'], 'id':x['Id']}

In [54]:
classparticipant_data['StartDate'] = classparticipant_data['StartDate'].apply(lambda x: x.strftime('%Y-%m-%d'))
classparticipant_data

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None
5,6,37,1,2020-03-16,None
6,7,38,1,2020-03-16,None
7,8,39,1,2020-03-16,None
8,9,40,1,2020-03-16,None
9,10,41,1,2020-03-16,None


In [43]:
print(classdict)
print()
print(studentdict)

{2.0: {'Section': 'GWDC201805DATA3', 'id': 'a083h0000016LjOAAU'}, 1.0: {'Section': 'GWU-ARL-DATA-PT-09-0', 'id': 'a083h0000016LjJAAU'}, 3.0: {'Section': 'GWARL201905WEB3', 'id': 'a083h0000016LjTAAU'}, 4.0: {'Section': 'GWARL201905UIUX3', 'id': 'a083h0000016LjAAAU'}}

{43.0: {'FirstName': 'Sarah', 'LastName': 'Schulte', 'id': 'a053h000006Nt2IAAS'}, 59.0: {'FirstName': 'Robert', 'LastName': 'Oppan', 'id': 'a053h000006Nt3aAAC'}, 60.0: {'FirstName': 'Trevor', 'LastName': 'Doucet', 'id': 'a053h000006Nt3fAAC'}, 57.0: {'FirstName': 'Michael', 'LastName': 'Clarke', 'id': 'a053h000006Nt3QAAS'}, 38.0: {'FirstName': 'Pariya', 'LastName': 'Poocharoen', 'id': 'a053h000006Nt1tAAC'}, 49.0: {'FirstName': 'Tom', 'LastName': 'Stark', 'id': 'a053h000006Nt2mAAC'}, 53.0: {'FirstName': 'Cole', 'LastName': 'Fingerut', 'id': 'a053h000006Nt36AAC'}, 62.0: {'FirstName': 'Kevin', 'LastName': 'Williamson', 'id': 'a053h000006Nt3pAAC'}, 51.0: {'FirstName': 'Chris', 'LastName': 'Mikus', 'id': 'a053h000006Nt2wAAC'}, 6

In [56]:
for index, row in classparticipant_data.iterrows():

    record = {"ID_ClassParticipant__c": row["ID_ClassParticipant"],
              "ID_Student__c": studentdict[row["ID_Student"]]['id'],
              "ID_Class__c": classdict[row["ID_Class"]]['id'],
              "StartDate__c": row["StartDate"],
              "EndDate__c": row["EndDate"],}
    
    try:
        sf.Class_Participant__c.create(record)
    except Exception as e:
        print(e)